In [86]:
# Source: https://www.turing.com/kb/deploy-ml-models-with-flask-and-docker

# Data Manipulation libraries
import pandas as pd
from sklearn.model_selection  import train_test_split
from sklearn.neural_network import MLPRegressor
import joblib

In [87]:
# import data
df = pd.read_csv('data/MELBOURNE_HOUSE_PRICES_LESS.csv')

#df = pd.read_csv('data/Melbourne_housing_FULL.csv')

# merge data
#df = pd.concat([df, df2], axis=0)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63023 entries, 0 to 63022
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         63023 non-null  object 
 1   Address        63023 non-null  object 
 2   Rooms          63023 non-null  int64  
 3   Type           63023 non-null  object 
 4   Price          48433 non-null  float64
 5   Method         63023 non-null  object 
 6   SellerG        63023 non-null  object 
 7   Date           63023 non-null  object 
 8   Postcode       63023 non-null  int64  
 9   Regionname     63023 non-null  object 
 10  Propertycount  63023 non-null  int64  
 11  Distance       63023 non-null  float64
 12  CouncilArea    63023 non-null  object 
dtypes: float64(2), int64(3), object(8)
memory usage: 6.3+ MB


In [88]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,Abbotsford,49 Lithgow St,3,h,1490000.0,S,Jellis,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
1,Abbotsford,59A Turner St,3,h,1220000.0,S,Marshall,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
2,Abbotsford,119B Yarra St,3,h,1420000.0,S,Nelson,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
3,Aberfeldie,68 Vida St,3,h,1515000.0,S,Barry,1/04/2017,3040,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,Airport West,92 Clydesdale Rd,2,h,670000.0,S,Nelson,1/04/2017,3042,Western Metropolitan,3464,10.4,Moonee Valley City Council


In [89]:
df.describe()

,Rooms,Price,Postcode,Propertycount,Distance
count,63023.000000,4.843300e+04,63023.000000,63023.000000,63023.000000
mean,3.110595,9.978982e+05,3125.673897,7617.728131,12.684829
std,0.957551,5.934989e+05,125.626877,4424.423167,7.592015
min,1.000000,8.500000e+04,3000.000000,39.000000,0.000000
25%,3.000000,6.200000e+05,3056.000000,4380.000000,7.000000
50%,3.000000,8.300000e+05,3107.000000,6795.000000,11.400000
75%,4.000000,1.220000e+06,3163.000000,10412.000000,16.700000
max,31.000000,1.120000e+07,3980.000000,21650.000000,64.100000


In [90]:
# drop na prices
df = df[df['Price'].notna()]

# get dummie for type
df = pd.get_dummies(df, columns=['Type'])

drop_cols = ['Suburb',
             'Address',
             'Method',
             'SellerG',
             'Date',
             'Postcode',
             'Regionname',
             'Propertycount',
             'CouncilArea',
             'Type_u'
]

df.drop(drop_cols, axis=1, inplace=True)



df.head()

,Rooms,Price,Distance,Type_h,Type_t
0,3,1490000.0,3.0,1,0
1,3,1220000.0,3.0,1,0
2,3,1420000.0,3.0,1,0
3,3,1515000.0,7.5,1,0
4,2,670000.0,10.4,1,0


In [91]:
X = df[['Rooms', 'Distance', 'Type_h', 'Type_t']]
y = df[['Price']]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 108)

# save the scaler
joblib.dump(scaler, "model/scaler.pkl")

['model/scaler.pkl']

In [92]:
X_train

,Rooms,Distance,Type_h,Type_t
43261,2.041217,0.701626,0.646364,-0.338536
13712,-0.075861,1.045999,0.646364,-0.338536
21472,-0.075861,0.211556,-1.547116,-0.338536
40046,-1.134401,-0.556661,-1.547116,-0.338536
25822,-2.192940,-0.530170,-1.547116,-0.338536
...,...,...,...,...
41554,-1.134401,-0.516925,0.646364,-0.338536
46446,0.982678,0.476459,0.646364,-0.338536
43467,-1.134401,-0.344739,-1.547116,2.953896
41105,2.041217,0.688380,0.646364,-0.338536


In [93]:
from sklearn.ensemble import RandomForestRegressor
regressor =  RandomForestRegressor() 
regressor.fit(X_train, y_train.values.ravel())

#predicting the test set result
y_pred = regressor.predict(X_test)#put results as a DataFrame

# RSME
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(rmse)

358405.33904872317


In [94]:
#Saving the machine learning model to a file
joblib.dump(regressor, "model/price_model.pkl")

['model/price_model.pkl']

In [95]:
X_test

,Rooms,Distance,Type_h,Type_t
21895,0.982678,1.589048,0.646364,-0.338536
20299,-1.134401,-0.649376,-1.547116,-0.338536
12980,0.982678,-0.861298,0.646364,-0.338536
10892,0.982678,0.701626,0.646364,-0.338536
6640,-1.134401,-1.205671,-1.547116,-0.338536
...,...,...,...,...
18606,-2.192940,-1.073220,-1.547116,-0.338536
19722,-0.075861,-0.755337,-1.547116,2.953896
28645,-2.192940,-0.172552,-1.547116,-0.338536
22139,-0.075861,-0.172552,-1.547116,-0.338536


In [96]:
y_pred

array([ 692401.69502195,  585244.27723482, 1684139.52963493, ...,
        313268.89388251,  870789.41159945, 1357334.90492077])

In [97]:
import json

data = {"Rooms":"3","Distance":"45","Type_h":1,"Type_t":0}

js = json.dumps(data)

df_json = pd.DataFrame(data, index=[0])

scaler = joblib.load('model/scaler.pkl')

x_scaled = scaler.transform(df_json)

x_scaled = pd.DataFrame(x_scaled, columns=df_json.columns)

x_scaled

,Rooms,Distance,Type_h,Type_t
0,-0.075861,4.277807,0.646364,-0.338536
